In [6]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim


REBUILD_DATA = False  # set to true to one once, then back to false unless you want to change something in your training data.

class DogsVSCats():
    IMG_SIZE = 50
    CATS = "/home/vikasunix/workspace/pytorch/PetImages/Cat"
    DOGS = "/home/vikasunix/workspace/pytorch/PetImages/Dog"
    LABELS = {CATS: 0, DOGS: 1}

    training_data = []
    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                    if label == self.CATS:
                        self.catcount+=1
                    elif label == self.DOGS:
                        self.dogcount += 1
                except Exception as e:
                    pass
        np.random.shuffle(self.training_data)
        np.save("training.npy", self.training_data)
        print("Cats", self.catcount)
        print("Dogs", self.dogcount)


if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()     

In [7]:

class Net(nn.Module):
    def __init__(self):
       super(). __init__()
       self.conv1 = nn.Conv2d(1, 32, 5)
       self.conv2 = nn.Conv2d(32, 64, 5)
       self.conv3 = nn.Conv2d(64, 128, 5)

       x = torch.randn(50,50).view(-1,1,50,50)
       self._to_linear = None
       self.convs(x)

       self.fc1 = nn.Linear(self._to_linear, 512)
       self.fc2 = nn.Linear(512, 2)

    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))

        print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

torch.Size([128, 2, 2])


In [8]:
training_data = np.load("training.npy", allow_pickle=True)
print("length of total data", len(training_data))


optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0 #Normalizing
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1 #validation percent
val_size = int(len(X)*VAL_PCT)
print("val size", val_size)

length of total data 24946
val size 2494


In [10]:
# train and test split
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

22452
2494


In [13]:
BATCH_SIZE = 100
EPOCHS = 1

def train(net):
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            net.zero_grad()

            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")


def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i])
            net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list, 
            predicted_class = torch.argmax(net_out)

            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))

In [14]:
train(net)

0%|          | 1/225 [00:00<00:49,  4.53it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
  7%|▋         | 15/225 [00:03<00:43,  4.81it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
  9%|▉         | 20/225 [00:04<00:45,  4.51it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 15%|█▌        | 34/225 [00:07<00:40,  4.75it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 21%|██▏       | 48/225 [00:10<00:39,  4.43it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 24%|██▎       | 53/225 [00:11<00:37,  4.53it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 24%|██▍       | 55/225 [00:11<00:35,  4.75it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 28%|██▊       | 63/225 [00:13<00:35,  4.56it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 29%|██▉       | 65/225 [00:13<00:32,  4.86it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 34%|███▍      | 77/225 [00:16<00:30,  4.78it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 37%|███▋      | 83/225 [00:17<00:29,  4.76

In [15]:
test(net)

rch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 71%|███████▏  | 1778/2494 [00:06<00:01, 424.82it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([12